In [1]:
import sys
sys.path.insert(0,'/home/berhe/Desktop/Thesis_git/TLP_thesis/codes/')
import Segmantation_lib as est
import Subtitle_truthvalue as stv
import evaluationMetrics as em
import pandas as pd
import os
from pyannote.metrics.diarization import DiarizationCoverage
import numpy as np

Subtitle files and Scene boundries files directory

In [2]:
scenes_dir='/home/berhe/Desktop/Thesis_git/TLP_thesis/Scenes/'
subtitle_dir='/home/berhe/Desktop/Thesis_git/TLP_thesis/subtitles/'

subtitleFiles=[]
for fl in os.listdir(subtitle_dir):
    if '.en.srt' in fl:
       subtitleFiles.append(fl)

scenesegementedFiles=[]
for fl in os.listdir(scenes_dir):
    if 'Episode' in fl:
       scenesegementedFiles.append(fl)
    
subtitleFiles.sort()
scenesegementedFiles.sort()

In [4]:
threads='/home/berhe/Desktop/Thesis_git/TLP_thesis/threadsShot/thread_output9_10_10.json'
shotSeq, shotBondry, _=est.preprocessingThreads(threads=threads)

In [5]:
scenesShotSequence=est.segmentation_speakers(shotSeq,7,4)

In [6]:
len(scenesShotSequence),len(shotSeq)

(33, 253)

In [7]:
sceneShotBoundry=[]
for i in scenesShotSequence:
    sceneShotBoundry.append(shotBondry[i])

start=sceneShotBoundry
start=[0.]+start
del start[-1]
len(start)

33

In [8]:
scenesegFile=scenes_dir+scenesegementedFiles[1]
subtFile=subtitle_dir+subtitleFiles[1]

In [9]:
_,_,sc_start,sc_end=stv.sceneTexts(scenesegFile,subtFile)

In [641]:
sceneShotBoundry[0:10],sc_start[0:3],(sc_end[0:3])

([5.24, 17.0, 22.44, 66.76, 72.44, 89.64, 102.88, 111.56, 119.56, 121.84],
 [0.0, 6000.0, 116100.0],
 [6000.0, 116100.0, 350000.0])

In [10]:
df=stv.get_text_time(subtFile)

In [15]:
df['textList'][101]

u'I never knew.'

In [643]:
len(df['start_seconds'])

732

In [644]:
df['textList'][1],df['end_second'][1],df['start_seconds'][1]

(u"(Varys) You've seen better days, my lord.", 125, 123)

In [645]:
scene_texts,scene_lbls,start,end=stv.sceneTexts(scenesegFile,subtFile)

In [646]:
len(scene_texts),len(scene_lbls),len(start),len(end)

(26, 26, 26, 26)

In [647]:
cl_sc=stv.split_into_sentences(scene_texts[2])

In [648]:
from nltk import sent_tokenize
cl_sc=sent_tokenize(scene_texts[10])

In [649]:
len(scene_texts[10]),len(cl_sc)

(275, 6)

In [650]:
episodeTVM,episodeTVMB,l1=stv.episodTruthValueMan1(scenesegementedFile=scenesegFile,subtitleFile=subtFile)
episodesTVA,episodesTVAB,l2=stv.episodTruthValueShot1(scene_start=start,scene_end=sceneShotBoundry,subtitleFile=subtFile)

In [651]:
len(episodesTVA),len(episodeTVM),l1,l2

(732, 732, 732, 732)

In [652]:
episodesTVAB[25:40],episodesTVA[25:40]

([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [653]:
em.pk(episodeTVM,episodesTVA,k=15,boundary=1)

0.479108635097493

In [654]:
em.windowdiff(episodeTVM,episodesTVA,k=13,boundary=1)

1.1015299026425591

In [655]:
episodeTVM.count(1)+1,episodesTVA.count(1)+1

(25, 61)

In [656]:
K=(len(episodeTVM)/(2*((episodeTVM.count(1)+1))))

In [657]:
K

14

Computing all evaluations for every different K and C values 

In [658]:
kvalues=[]
cvalues=[]
windiffSHotA=[]
pkShotA=[]
shotRecA=[]
shotPrA=[]
shotRecB=[]
shotPrB=[]
sceneShot=[]
PurityB=[]
NMIAB=[]
Purity=[]
NMIA=[]
Coverage=[]
Coverage1=[]
Coverage2=[]
Coverage3=[]
for k in range(4, 11):
    for c in range(2,9):
        if c<k:
            scenesShotSequence=est.segmentation_speakers(shotSeq,k,c)
            sceneShotBoundry=[]
            for i in scenesShotSequence:
                sceneShotBoundry.append(shotBondry[i])

            start=sceneShotBoundry
            start=[0.]+start
            del start[-1]
            len(start)
            
            episodeTVM,episodeTVMB,l1=stv.episodTruthValueMan1(scenesegementedFile=scenesegFile,subtitleFile=subtFile)
            episodeTVA,episodeTVAB,l2=stv.episodTruthValueShot1(scene_start=start,scene_end=sceneShotBoundry,subtitleFile=subtFile)
            
            
            windiffSHot=round(em.windowdiff(episodeTVM,episodeTVA,k=K,boundary=1),2)
            pkShot=round(em.pk(episodeTVM,episodeTVA,k=K,boundary=1),2)
            
            episodeTVA=[int(i) for i in episodeTVA]
            episodeTVM=[int(i) for i in episodeTVM]
            episodeTVAB=[int(i) for i in episodeTVAB]
            episodeTVMB=[int(i) for i in episodeTVMB]
            episodeTVA=np.array(episodeTVA)
            episodeTVM=np.array(episodeTVM)
            episodeTVAB=np.array(episodeTVAB)
            episodeTVMB=np.array(episodeTVMB)
                
            shotRec=round(em.recall(episodeTVM,episodeTVA),3)
            shotPr=round(em.precision(episodeTVM,episodeTVA),3)
            shotRecb=round(em.recall(episodeTVMB,episodeTVAB,rType='micro'),3)
            shotPrb=round(em.precision(episodeTVMB,episodeTVAB,rType='micro'),3)
            purityb=round(em.purity_score(y_true=episodeTVMB,y_pred=episodeTVAB),3)
            nmib=round(em.nmi(episodeTVMB,episodeTVAB),3)
            purity=round(em.purity_score(y_true=episodeTVM,y_pred=episodeTVA),3)
            nmie=round(em.nmi(episodeTVM,episodeTVA),3)
            #coverage=DiarizationCoverage()
            dc=round(em.coverage(y_true=episodeTVM,y_pred=episodeTVA),3)
            dc1=round(em.coverage(y_true=episodeTVMB,y_pred=episodeTVAB),3)
            dc2=round(em.purity_score(y_true=episodeTVA,y_pred=episodeTVM),3)
            dc3=round(em.purity_score(y_true=episodeTVAB,y_pred=episodeTVMB),3)
                
            
            kvalues.append(k)
            cvalues.append(c)
            windiffSHotA.append(windiffSHot)
            pkShotA.append(pkShot)
            shotRecA.append(shotRec)
            shotRecB.append(shotRecb)
            PurityB.append(purityb)
            NMIAB.append(nmib)
            Purity.append(purity)
            NMIA.append(nmie)
            shotPrA.append(shotPr)
            shotPrB.append(shotPrb)
            sceneShot.append(len(scenesShotSequence))
            Coverage.append(dc)
            Coverage1.append(dc1)
            Coverage2.append(dc2)
            Coverage3.append(dc3)
        else:
            break

Representing the Evaluation as a table (dataframe)

In [659]:
resulDF=pd.DataFrame()
resulDF['K']=kvalues
resulDF['C']=cvalues
resulDF['wdSH']=windiffSHotA
resulDF['pkSh']=pkShotA
#resulDF['rSh']=recallShot
resulDF['shotRec_01']=shotRecB
resulDF['shotPr_01']=shotPrB
resulDF['shotRec']=shotRecA
resulDF['shotPr']=shotPrA
resulDF['sceneShot']=sceneShot
resulDF['purity']=Purity
resulDF['NMI']=NMIA
resulDF['purity_01']=PurityB
resulDF['NMI_01']=NMIAB
resulDF['Coverage']=Coverage
resulDF['Coverage1']=Coverage1
resulDF['CoverageP']=Coverage2
resulDF['CoverageP1']=Coverage3

In [660]:
resulDF

,K,C,wdSH,pkSh,shotRec_01,shotPr_01,shotRec,shotPr,sceneShot,purity,NMI,purity_01,NMI_01,Coverage,Coverage1,CoverageP,CoverageP1
0,4,2,1.53,0.48,0.004,0.004,0.167,0.048,85,0.967,0.002,0.787,0.762,0.885,0.628,0.885,0.628
1,4,3,0.84,0.42,0.004,0.004,0.208,0.109,47,0.967,0.022,0.746,0.783,0.937,0.732,0.937,0.732
2,5,2,1.53,0.48,0.004,0.004,0.167,0.048,85,0.967,0.002,0.787,0.762,0.885,0.628,0.885,0.628
3,5,3,0.84,0.42,0.004,0.004,0.167,0.089,46,0.967,0.013,0.716,0.771,0.939,0.768,0.939,0.768
4,5,4,1.16,0.48,0.004,0.004,0.167,0.066,34,0.967,0.006,0.653,0.743,0.917,0.768,0.917,0.768
5,6,2,1.53,0.48,0.004,0.004,0.167,0.048,85,0.967,0.002,0.787,0.762,0.885,0.628,0.885,0.628
6,6,3,0.83,0.44,0.004,0.004,0.208,0.114,45,0.967,0.024,0.706,0.772,0.940,0.736,0.940,0.736
7,6,4,1.16,0.48,0.004,0.004,0.167,0.066,34,0.967,0.006,0.653,0.743,0.917,0.768,0.917,0.768
8,6,5,0.61,0.41,0.004,0.004,0.167,0.138,27,0.967,0.027,0.649,0.769,0.960,0.818,0.960,0.818
9,7,2,1.53,0.48,0.004,0.004,0.167,0.048,85,0.967,0.002,0.787,0.762,0.885,0.628,0.885,0.628


In [634]:
resulDF.loc[resulDF['purity'].idxmax()]

K               4.000
C               2.000
wdSH            3.290
pkSh            0.560
shotRec_01      0.004
shotPr_01       0.004
shotRec         0.500
shotPr          0.066
sceneShot     191.000
purity          0.967
NMI             0.017
purity_01       0.919
NMI_01          0.753
Coverage        0.750
Coverage1       0.404
CoverageP       0.750
CoverageP1      0.404
Name: 0, dtype: float64

In [31]:
resulDF.loc[resulDF['pkSh'].idxmin()]

K              7.000
C              6.000
wdSH           0.740
pkSh           0.370
shotRec_01     0.004
shotPr_01      0.004
shotRec        0.231
shotPr         0.133
sceneShot     45.000
purity         0.967
NMI            0.032
purity_01      0.767
NMI_01         0.824
Coverage       0.942
Coverage1      0.795
CoverageP      0.942
CoverageP1     0.795
Name: 3, dtype: float64

In [32]:
resulDF.max(axis=None, skipna=None, level=None, numeric_only=None)

K              10.000
C               8.000
wdSH            1.790
pkSh            0.510
shotRec_01      0.014
shotPr_01       0.014
shotRec         0.346
shotPr          0.150
sceneShot     113.000
purity          0.967
NMI             0.038
purity_01       0.843
NMI_01          0.824
Coverage        0.959
Coverage1       0.795
CoverageP       0.959
CoverageP1      0.795
dtype: float64

In [33]:
labels_true = episodeTVM
labels_true2 = episodeTVMB
#[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,]
labels_pred = episodeTVA
labels_pred2 = episodeTVAB
#[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
 #0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels_pred=np.array(labels_pred)
labels_true=np.array(labels_true)
labels_pred2=np.array(labels_pred2)
labels_true2=np.array(labels_true2)

In [34]:
em.purity_score(y_true=labels_true,y_pred=labels_pred),em.purity_score(y_true=labels_true2,y_pred=labels_pred2)

(0.9667093469910372, 0.678617157490397)

In [35]:
em.coverage(y_true=labels_true,y_pred=labels_pred),em.coverage(y_true=labels_true2,y_pred=labels_pred2)

(0.9590268886043534, 0.7708066581306018)

In [36]:
em.purity_score(y_true=labels_pred,y_pred=labels_true),em.purity_score(y_true=labels_pred2,y_pred=labels_true2)

(0.9590268886043534, 0.7708066581306018)

In [38]:
em.purity_score(y_true=labels_pred,y_pred=labels_pred),em.coverage(y_true=labels_pred2,y_pred=labels_pred2)

(1.0, 1.0)